# Melody Generation

+ **AI in Culture and Arts - Tech Crash Course**
+ **Date:** 06.06.2024
+ **Author:** B. Zönnchen

<a href="https://colab.research.google.com/github/aica-wavelab/aica-assignments/blob/main/A4_melody_generation/2_1_genuine_composing.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In the following we will create music sheets and sound. For those tasks ``Python`` requires external programs that you should install if you are working locally:

1. [Musescore](https://musescore.org/de) (for generating sheets)
2. [FluidSynth](https://www.fluidsynth.org/) (for generating sound)

If you are working on google ``Colab`` you can evaluate the following three cells to install these applications:

In [ ]:
#@title install dependencies to play sound
%%capture
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
print('done!')

In [ ]:
#@title install dependencies to show score in music notation
%%capture
print('installing musescore3...')
!apt-get install musescore3 > /dev/null
print('done!')

In [ ]:
#@title clone git repository
%%capture
!rm -rf musical-interrogation
!git clone https://github.com/aica-wavelab/aica-assignments.git
%cd A4_melody_generation

Furtheremore, for this notebook, we need the following ``Python`` packages and moduls. Execute the cell to install them:

In [ ]:
%pip install music21
%pip install pyfluidsynth
%pip install otter-grader

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("2_1_genuine_composing.ipynb")

In [ ]:
import random
import music21 as m21
from music21.note import Note, Rest
from music21.chord import Chord
from music21.stream import Part, Score

# 2 Simple Melody Generation

## 2.2 Genuine Composition

We have now the ability to create musical (symolical) pieces via code. In principle we can compose music algorithmically.

### 2.1 Pure Randomness

We can create a function which generates a purely random ``Score`` without considering any music thery with the exception that we use notes:

In [ ]:
def random_score(length=10, lower=m21.note.Note('C3'), upper=m21.note.Note('C6'), q=0.1):
  score = Score()
  part = Part()

  for _ in range(length):
    
    # with probability q there will be a rest
    if random.random()<=q:
      part.append(Rest())
    else:
      midi = random.randint(lower.pitch.midi, upper.pitch.midi)
      note = Note()
      note.pitch.midi = midi
      part.append(note)
  score.insert(0, part)
  return score

Let us listen to the result. It sounds quite unmusical and random.

In [ ]:
score = random_score(30)
score.show('midi')

<div class="alert alert-info">

**Instruction 2.1.1** Write a function ``random_cmajor_score(length, q=0.1)`` that generates a random ``Score`` (a melody) in the key of C major of ``length`` using only one octave. ``length`` determines the number elements (``Rest``s or ``Note``s) of the ``Score``. You can use ``random.randint(a, b)`` to generate random whole numbers (i.e. integers). You might wanna pick certain durations and you might want to include ``Rest``s. ``q`` can be the probability that a ``Rest`` will be 'played'. Test your result, that is, generate a piece of length ``n`` and play it back.

</div>

In [ ]:

def random_cmajor_score(length=10, q=0.1):
  score = Score()
  ...
  return score

score = random_cmajor_score(20)
score.show('midi')
score.recurse().notesAndRests

In [ ]:
grader.check("q21")

### 2.2 Exploiting Music Theory

A melody that is generated purely randomly without any considerations to music theory doesn't sound great. It sounds random. Therefore, let us talk a little bit about music theory.

We say that two notes form an interval and it is this interval we have to look at to see if two notes are rather *consonant* or *disonant*. The pitch is defined by a freuency which measures how often a certain pattern is repeated. For example, the pitch ``C4`` represents (depending on the how an instrument is tuned) a frequency equal to

In [ ]:
Note("C4").pitch.frequency

Therefore, if something oscillates with a frequency of 261.625 Hz (repetitions per second), then we perceive a ``C4``. This perception is of course subjective. Doubling the frequency leads to a pitch one octave higher:

In [ ]:
print(Note("C5").pitch.frequency) # this is roughly Note("C4").pitch.frequency * 2
print(Note("C4").pitch.frequency * 2) 

This phenomenon, that is, the octave relationship is a natural phenomenon that has been referred to as the *basic miracle of music*. Its use is common in most musical systems.

If we consider two frequencies, we get some interference since they form a new pattern that repeats depending on both frequencies. For example if our first frequency is equal to $f_1$ and our second one is equal to 

$$f_2 = f_1 \cdot 2$$

then their ratio is simple:

$$\frac{f_1}{f_2} = \frac{1}{2}.$$

In this case, if both pattern start at the same time, the second one repeats the second time when the first one finishes the first time. Furthermore, there is a new repetition determined by the second pattern. We say that $f_2$ is a *harmonic* of its *fundamental frequency* $f_1$. In fact, hitting a string generates the harmonic series:

$$\sum_{k=0}^N f_1 \cdot 2^k.$$

Therefore, playing ``C4`` and ``C5`` together or in sequence sounds consonant and rather pleasing (but also boring).
Let us play ``C4`` and ``C5`` together. We can use the ``Chord`` object to do this:

In [ ]:
chord = Chord(['C4', 'C5'])
# this interval sounds very consonant
chord.show('midi')

On the other hand, there are intervals that sound disonant. Consider ``C4`` and the pitch 6 semitones above, that is, ``F#`` or ``G-``. This interval is sometimes called *the Devil's Tritone* because it is very disonant.
Let's have a look at the respective frequencies:

In [ ]:
print(f'C4 frequncy: {Note("C4").pitch.frequency}') # this is roughly Note("C4").pitch.frequency * 2
print(f'Gb frequncy: {Note("G4-").pitch.frequency}') 
print(f'ratio: {Note("G4-").pitch.frequency/Note("C4").pitch.frequency}')
print(f'square root of 2 = {2**0.5}')

The ratio is approximately ``1.414`` which is close to $\sqrt{2}$ which is an irrational number! Therefore, the resulting pattern emerging when playing both notes repeats hardly ever.
Thus we perceive it as *dissonant*.

In [ ]:
chord = Chord(['C4', 'G4-'])
# this interval sounds very dissonant
chord.show('midi')

However, our ears are not that perfect and if a ratio is close to a simple ratio, we still perceive it as *consonant*. In fact, most Western pianos follow the *twelve-tone equal temperament tuning* which was first introduced in China in 1584. 

There are ``12`` different tones in each octave and the octave interval is ``2``. To be able to play a piece of music in multiple keys or to make it easier to change keys during a piece, the *twelve-tone equal temperament tuning* divides the octave evenly! Therefore, in this system, each interval of two consecutive notes e.g. ``E4`` and ``F4`` or ``F4`` and ``F4#`` is equal. Thus all these intervals have to be (or very close to)

$$2^{\frac{1}{12}} = \sqrt[12]{2}$$

because

$$2^{\frac{1}{12} \cdot 12} = 2.$$

However, $2^{\frac{1}{12}}$ is an irrational number! Therefore, most inverals are irrational as well. For example, we the interval formed by ``C4`` ($f_1$) and ``F4`` ($f_2$) to be very *consonant*. This interval is called the *perfect fifth*. It spans 7 semitones thus

$$f_2 = f_1 \cdot 2^{\frac{1}{12} \cdot 7} = 2^{\frac{7}{12}} = 1.4930...$$

But this number is close to 1.5 and therefore we perceive it as very *consonant*. On the other hand, $\sqrt{2}$ is close to 64/45 which is not a simple ratio, therefore we perceive it as *dissonant*.

In summary, adding a semitone to a note is equal to multiplying its frequency by $2^\frac{1}{12}$:

In [ ]:
note =Note('C4')
frequency = note.pitch.frequency
print(f'frequency of C4 {note.pitch.frequency}')

note.pitch.midi += 1
print(f'frequency of C4 + 1 semitone {note.pitch.frequency}')
print(f'frequency times 2 to the power of 1/12: {frequency * (2**(1/12))}')

Very generally speaking, many regard the interval of ``0``, ``12``, ``7``, ``5`` to be very *consonant*.

In [ ]:
tonic_note = Note('C4')

octave_above = Note('C4')
octave_above.pitch.midi += 12

perfect_fifth_above = Note('C4')
perfect_fifth_above.pitch.midi += 7

perfect_fourth_above = Note('C4')
perfect_fourth_above.pitch.midi += 5

octave = Chord([tonic_note, octave_above])
perfect_fifth = Chord([tonic_note, perfect_fifth_above])
perfect_fourth = Chord([tonic_note, perfect_fourth_above])

# this interval sounds very dissonant
part = Part()
part.append([octave, perfect_fifth, perfect_fourth])
part.show('midi')

Often chords are inverted, meaning we can shift notes within a chord up and down one or even multiple octaves and they still sound pleasing if they are consonant in the root position, because the ratio will be multiplied or divided by ``2`` which will result in a simple ratio if the ratio was simple before.

In [ ]:
tonic_note = Note('C4')

octave_above = Note('C4')
octave_above.pitch.midi += 12 + 12

perfect_fifth_above = Note('C4')
perfect_fifth_above.pitch.midi += 7 + 12

perfect_fourth_above = Note('C4')
perfect_fourth_above.pitch.midi += 5 + 12

octave = Chord([tonic_note, octave_above])
perfect_fifth = Chord([tonic_note, perfect_fifth_above])
perfect_fourth = Chord([tonic_note, perfect_fourth_above])

# this interval sounds very dissonant
part = Part()
part.append([octave, perfect_fifth, perfect_fourth])
part.show('midi')

<div class="alert alert-info">

**Instruction 2.1.2** Write a function ``semi_random_score(note, length)`` that generates a random ``Score`` (a melody) in the key of C major of length ``length`` but this time integrate some music theory knowledge and use only certain intervals.

</div>

In [ ]:
def semi_random_score(tonic_note=m21.note.Note('C4'), length=10):
  score = Score()
  ...
  return score

score = semi_random_score(tonic_note=m21.note.Note('C4'), length=20)
score.show('midi')


You just managed to generate a **genuine composition**! Next we want to use **style imitation** to compose new melodies using *machine learning (ML)*. Suppose you have a corpus of melodies, that is, monophonic pieces and you want to generate new pieces that imitate the style of the given corpus. One very simple approach is to compute the frequency of appearance of events which are defined by any possible consecutive pairs of notes (including ``Rest``s) appearing in the pieces of the corpus.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)